<a href="https://colab.research.google.com/github/nematatu/Colab-Python/blob/master/Hello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('Hello, World!')

Hello, World!


In [2]:
print("anyo--n!")

anyo--n!
